In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df= pd.read_pickle('/content/drive/MyDrive/Data_M/clean_data.txt')

In [6]:
df.head()

,cleaned_tweet
0,être avoir culot venir jouer victime méchant g...
1,jslalsbd faisce veu dieuj llook effectvuemy ce...
2,unisex
3,meme être fan puma aller vaincre milan ac team...
4,regarde truc audessus savoir


In [7]:
cleaned_tweet=df.cleaned_tweet

## 6 - **Vectorization** (featurization)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

A- **Méthode BOW**

In [9]:
# définir le tokenizer
def tokenizer(text):
    return text


In [10]:
bow_vectorizer = CountVectorizer(analyzer='word',lowercase=False)
print('Tweet before vectorization: {}'.format(cleaned_tweet[83]))
bow = bow_vectorizer.fit_transform(cleaned_tweet)

print('Tweet after vectorization: \n{}'.format(bow[83]))
bow.shape

Tweet before vectorization: monsieur député attention souffrance peuple iranien démocratie pouvoir apporter paix pays cnri soutenir œuvre obtenir
Tweet after vectorization: 
  (0, 7977)	1
  (0, 701)	1
  (0, 7556)	1
  (0, 6674)	1
  (0, 3269)	1
  (0, 9602)	1
  (0, 7662)	1
  (0, 5268)	1
  (0, 3224)	1
  (0, 503)	1
  (0, 7392)	1
  (0, 1950)	1
  (0, 9630)	1
  (0, 11413)	1
  (0, 7164)	1


(5000, 11452)

In [11]:
#Mots les plus courants et leur nombre respectif
def get_most_freq_words(str, n=None):
    vect = CountVectorizer().fit(cleaned_tweet)
    bag_of_words = vect.transform(cleaned_tweet)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for tweet in cleaned_tweet for word in tweet],20)

[('être', 2539),
 ('avoir', 1727),
 ('faire', 525),
 ('aller', 359),
 ('pouvoir', 239),
 ('voir', 205),
 ('bien', 196),
 ('savoir', 149),
 ('jour', 136),
 ('vouloir', 131),
 ('france', 118),
 ('falloir', 118),
 ('devoir', 117),
 ('gagner', 105),
 ('amp', 99),
 ('vie', 98),
 ('follow', 94),
 ('prendre', 91),
 ('venir', 88),
 ('grand', 88)]

J'essaie ici de jouer avec min_df et max_df de countvectorizer , et ça n'a pas l'air de marcher , car les tweets ont bien été nettoyé, de plus cela témoingne d'une grande diversité des mots au sein de notre corpus

In [12]:
#Mots les plus courants et leur nombre respectif
def get_most_freq_words(str, n=None):
    vect = CountVectorizer(max_df=0.3, min_df=5).fit(cleaned_tweet)
    bag_of_words = vect.transform(cleaned_tweet)
    sum_words = bag_of_words.sum(axis=0) 
    freq = [(word, sum_words[0, idx]) for word, idx in vect.vocabulary_.items()]
    freq =sorted(freq, key = lambda x: x[1], reverse=True)
    return freq[:n]
  
get_most_freq_words([ word for tweet in cleaned_tweet for word in tweet],20)

[('avoir', 1727),
 ('faire', 525),
 ('aller', 359),
 ('pouvoir', 239),
 ('voir', 205),
 ('bien', 196),
 ('savoir', 149),
 ('jour', 136),
 ('vouloir', 131),
 ('france', 118),
 ('falloir', 118),
 ('devoir', 117),
 ('gagner', 105),
 ('amp', 99),
 ('vie', 98),
 ('follow', 94),
 ('prendre', 91),
 ('venir', 88),
 ('grand', 88),
 ('semaine', 86)]

On constate ici que les mots les plus courants avec BOW sont des verbes non conjugés tels que : être, faire, avoir, etc. 

B- **Méthode TFIDF**

In [13]:
#from numpy.lib.function_base import vectorize
#utiliser la méthode de Tfidfvectorizer avec:
""" - les stopsword français importés depuis spacy
    - le tokenizer définis plus haut"""
vectorizer = TfidfVectorizer(lowercase=False)
print('Tweet before vectorization: {}'.format(cleaned_tweet[83]))

Tweet before vectorization: monsieur député attention souffrance peuple iranien démocratie pouvoir apporter paix pays cnri soutenir œuvre obtenir


In [14]:
# transformer les tweets en vecteurs
# qui prend en argument les vecteurs de mots de l'ensemble des tweets nettoyés précédemment 
X = vectorizer.fit_transform(cleaned_tweet)
print('Tweet after vectorization: \n{}'.format(X[83]))

#print ((X.todense()))

Tweet after vectorization: 
  (0, 7164)	0.26651956770453816
  (0, 11413)	0.2622991202871934
  (0, 9630)	0.21262488304697705
  (0, 1950)	0.31619380494475446
  (0, 7392)	0.28336092004478347
  (0, 503)	0.28336092004478347
  (0, 3224)	0.27130431635697344
  (0, 5268)	0.31619380494475446
  (0, 7662)	0.2286786786768661
  (0, 9602)	0.28336092004478347
  (0, 3269)	0.2491226633388327
  (0, 6674)	0.2491226633388327
  (0, 7556)	0.19257251307930415
  (0, 701)	0.25510861009461533
  (0, 7977)	0.14809160802751328


In [15]:
idf=vectorizer.idf_

C- **Word embeding with Spacy**

In [16]:
def my_word_embedding(sentence):
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le vecteur lié à chaque token
    return [(X.vector) for X in doc]

#for n, snt in enumerate(s):
    #print(n , ":", my_word_embedding(snt), "\n")

    print(X)

D- **Méthode W2Vec**

In [29]:
data=list(cleaned_tweet)
cleaned_tweet_list= [line.split() for line in data]

In [30]:
import gensim


#tokenized_tweet = s.apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            cleaned_tweet_list,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 


In [31]:
print(model_w2v)

Word2Vec(vocab=4615, size=200, alpha=0.025)


In [32]:
words = list(model_w2v.wv.vocab)
print(words)

['être', 'avoir', 'venir', 'jouer', 'victime', 'méchant', 'gros', 'compte', 'lance', 'vague', 'abonné', 'insupportable', 'voir', 'çer', 'faire', 'direct', 'veu', 'cest', 'mets', 'pa', 'pression', 'tablette', 'meme', 'fan', 'aller', 'vaincre', 'milan', 'ac', 'om', 'regarde', 'truc', 'audessus', 'savoir', 'file', 'minute', 'trottoir', 'quartier', 'alentours', 'bordeaux', 'chef', 'lieu', 'grand', 'région', 'administration', 'boîte', 'privé', 'no', 'es', 'devoir', 'mémoire', 'reprendre', 'vie', 'grâce', 'olivier', 'laurent', 'article', 'info', 'extrême', 'droite', 'rentrer', 'cas', 'exactemer', 'reproche', 'gauche', 'préférer', 'soigner', 'aide', 'soignant', 'perso', 'prendre', 'charge', 'manqu', 'personnel', 'nonvacciné', 'hôpital', 'irresponsable', 'tchau', 'département', 'policier', 'intensifier', 'contrôle', 'passsanitair', 'établissement', 'recevoir', 'public', 'midi', 'centre', 'ville', 'lutter', 'covid', 'vue', 'saisissant', 'fort', 'mont', 'couleur', 'ilovenice', 'nice', 'cotedazur

In [33]:
#Nombre de mots dans le vocabulaire 

vocab_len = len(model_w2v.wv.vocab)
print(vocab_len)

4615


In [ ]:
model_w2v['haine']

In [41]:
model_w2v.wv.most_similar(positive="amour")

[('envier', 0.9998581409454346),
 ('wsh', 0.9998572468757629),
 ('réfléchir', 0.999854326248169),
 ('artiste', 0.9998530149459839),
 ('contraire', 0.999851644039154),
 ('type', 0.9998509287834167),
 ('main', 0.9998493790626526),
 ('lever', 0.9998488426208496),
 ('aider', 0.9998481869697571),
 ('magazine', 0.9998480081558228)]